In [2]:
import json
with open('intents.json', 'r', encoding='utf-8') as json_data:
    intents = json.load(json_data)

In [3]:
intents

{'intents': [{'tag': 'greeting', 'patterns': ['Привет', 'Как дела?', 'Здорова', 'Есть кто?'], 'responses': ['Приветствую', 'Чем могу помочь?', 'Здравствуйте'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['Пока', 'До свидания'], 'responses': ['Спасибо за обращение', 'Хорошего дня']}, {'tag': 'thanks', 'patterns': ['Спасибо', 'Благодарю'], 'responses': ['Рада помочь', 'С Удовольствием', 'В любое время']}, {'tag': 'orange', 'patterns': ['Трунь'], 'responses': ['Чпок']}]}

In [4]:
import nltk
nltk.download('punkt')
nltk.download("stopwords")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Win10Pro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Win10Pro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [6]:
X = []
y_text = []

In [7]:
# Tokenize and stemming each pattern sentence
for doc in intents['intents']:
    for p in doc['patterns']:
        X.append(" ".join([stemmer.stem(i) for i in nltk.word_tokenize((p.lower()))]))
        y_text.append(doc['tag'])

In [8]:
X

['привет', 'как дела ?', 'здорова', 'есть кто ?', 'пока', 'до свидания', 'спасибо', 'благодарю', 'трунь']

In [9]:
y_text

['greeting', 'greeting', 'greeting', 'greeting', 'goodbye', 'goodbye', 'thanks', 'thanks', 'orange']

In [10]:
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words = russian_stopwords, analyzer = 'word')

In [12]:
X_cv = cv.fit_transform(X)

In [13]:
X_cv.shape

(9, 8)

In [14]:
from sklearn.preprocessing import LabelEncoder
label_encoder_y = LabelEncoder()
y = label_encoder_y.fit_transform(y_text)

In [15]:
y

array([1, 1, 1, 1, 0, 0, 3, 3, 2], dtype=int64)

In [16]:
y_mapping = {}
for i in range(len(y_text)):
    if y_text[i] not in y_mapping:
        y_mapping[y[i]] = y_text[i]

In [17]:
y_mapping

{1: 'greeting', 0: 'goodbye', 3: 'thanks', 2: 'orange'}

In [18]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier()

In [19]:
random_forest.fit(X_cv, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [20]:
test_data = "Трунь"
test_data = " ".join([stemmer.stem(i) for i in nltk.word_tokenize((test_data.lower()))])

In [21]:
X_cv_test = cv.transform([test_data])
result = random_forest.predict(X_cv_test)

In [22]:
random_forest.predict_proba(X_cv_test)

array([[0.04, 0.22, 0.71, 0.03]])

In [23]:
predicted_tag = y_mapping[int(result)]

In [24]:
import random
for doc in intents['intents']:
    if doc['tag'] == predicted_tag:
        print(random.choice(doc['responses']))

Чпок
